In [1]:
#meta 9/17/2021 ALBERT BASIC USAGE
# during work on Inference portion with BERT prj HS6 Clf POC 
# task: ultimately classify text, here embedding portion only
# input: pre-trained BERT models
# infra: Kaggle kernel
# previously infra AML CI: anya-ml-ci-gpu-tf25
# previously env: py38_tf25_gpu

#related to p2_hs6_nlp_classify_with_bert_infe

# 9/21/2021 KAGGLE KERNEL.ipynb

# ALBERT Basic Usage
## albert_en_base
ALBERT: A Lite BERT for Self-supervised Learning of Language Representations

##### Usage
This `SavedModel` implements the encoder API for text embeddings with transformer encoders (src https://www.tensorflow.org/hub/common_saved_model_apis/text#transformer-encoders). It expects a dict with three int32 Tensors as input: `input_word_ids`, `input_mask`, and `input_type_ids`.

The separate `preprocessor SavedModel` at http://tfhub.dev/tensorflow/albert_en_preprocess/3 transforms plain text inputs into this format, which its documentation describes in greater detail.

In [2]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

tf.get_logger().setLevel('ERROR')

In [3]:
#$xtra ALBERT
# ALBERT: A Lite BERT for Self-supervised Learning of Language Representations
# src https://tfhub.dev/tensorflow/albert_en_base/2

#basic usage
# The simplest way to use this model in the Keras functional API is src: https://www.tensorflow.org/guide/keras/functional
# The encoder's outputs are the pooled_output to represents each input sequence as a whole, and the sequence_output to represent each input token in context. 
# Either of those can be used as input to further model building.

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer(
    "http://tfhub.dev/tensorflow/albert_en_preprocess/3")
encoder_inputs = preprocessor(text_input)
encoder = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/albert_en_base/2",
    trainable=True)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 768].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 768].

In [6]:
#$xtra Load ALBERT cont.
# To print pooled_outputs for inspection
embedding_model = tf.keras.Model(text_input, pooled_output)
sentences = tf.constant(["(textile product 511 jeans)"])
print(embedding_model(sentences))

tf.Tensor(
[[-1.39235333e-01  1.82157680e-01  1.04073090e-02 -1.19054168e-01
   6.18295372e-01 -9.92320538e-01 -7.71006718e-02  1.70437861e-02
   6.80912361e-02 -8.47790301e-01  3.56587231e-01  1.28351534e-02
   7.62324631e-01 -6.76137507e-01 -5.83042264e-01 -6.40809089e-02
  -1.95365563e-01 -1.44290417e-01  9.58460212e-01  1.27124250e-01
  -3.55047315e-01 -9.94184554e-01  9.91895795e-01  9.15349543e-01
   2.13481218e-01  1.66672125e-01  3.27845244e-03 -9.99818146e-01
  -9.83971894e-01  7.12293237e-02 -9.99730587e-01  1.59588009e-02
  -2.43775807e-02 -1.30286008e-01  9.24877077e-02  3.82654294e-02
  -4.65728790e-02 -1.11131832e-01 -2.33145989e-02 -1.33705959e-01
   7.41439983e-02 -9.83656049e-01 -6.33449614e-01 -7.07468167e-02
  -9.23860893e-02  2.69552711e-02  9.04321432e-01 -4.30620611e-01
   8.90949547e-01  1.52917162e-01  2.17329904e-01 -3.57523076e-02
   2.57466715e-02 -8.69128466e-01 -3.92253734e-02 -1.86108261e-01
  -2.10220695e-01  1.40194237e-01  9.98371542e-01 -4.30888236e-01

In [7]:
embedding_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_word_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'sequence_output':  11683584    keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
Total params: 11,683,584
Trainable params: 11,683,584
Non-trainable params: 0
______________